In [12]:
import callhorizons
import datetime
import os
import subprocess
import time
from astropy import units as u
from astropy.coordinates import Angle
import logging
from logging.handlers import RotatingFileHandler
import unicodedata
import string
import re
import pytz
import time

In [13]:
#the datetimes
#Aug.  7 Mo  02:30 06:15         X  023 Nowinski             SpeX/MORIS           SJB First Night
#Oct.  9 Mo  00:05 05:55         X  023 Nowinski             SpeX/MORIS           SJB 
#Nov. 25 Sa  00:35 05:55         X  023 Nowinski             SpeX/MORIS           SJB
#Nov. 26 Su  18:35 01:05         X  023 Nowinski             SpeX/MORIS           SJB 
#Dec. 29 Fr  18:45 23:20         X  023 Nowinski             SpeX/MORIS           ACB
#convert to UTC (+10)
dts = [ [datetime.datetime(2017, 8, 7, 12, 30, 0), datetime.datetime(2017, 8, 7, 16, 15, 0)],
        [datetime.datetime(2017, 10, 9, 10, 5, 0), datetime.datetime(2017, 10, 9, 15, 55, 0)],
        [datetime.datetime(2017, 11, 25, 10, 35, 0), datetime.datetime(2017, 11, 25, 15, 55, 0)],
        [datetime.datetime(2017, 11, 27, 4, 35, 0), datetime.datetime(2017, 11, 27, 11, 5, 0)],
        [datetime.datetime(2017, 12, 30, 4, 45, 0), datetime.datetime(2017, 12, 30, 9, 20, 0)]       
      ]

In [14]:
fname_targets = 'targets.txt'
fname_observables = 'observables.txt'

#the target using spkid
#targets = ['2009633']
targets = open(fname_targets, "r").readlines()

#output file
f = open(fname_observables, "w")
f.write("spkid,name,minutes\n")
f.close()

In [15]:
#the observatory
observatory = '568'

In [16]:
#min elevation in degrees
min_elev = 30;

#min minutes observable per night
min_minutes = 90;

#faintest apparent magnitude of target
faintest_apmag = 18;

In [18]:
#get ephemerides for targets in JPL Horizons from start to end times
#for spkid, use DES=spkid; for target name
#loop thru all targets
for target in targets:
    days_available = 0
    total_minutes = 0
    ch=callhorizons.query('DES='+target.strip()+";", smallbody=True)
    print 'Processing %s...'%(target.strip())
    #f = open(fname_observables, "a")	
    #f.write('#Processing %s...\n'%(target.strip()))
    #f.close()
    #loop thru all datetimes
    for dt in dts:
        #get ephemerides for specified observation windows in 1m increments
        ch.set_epochrange(dt[0].strftime("%Y/%m/%d %H:%M:%S"), dt[1].strftime("%Y/%m/%d %H:%M:%S"), '1m')
        ch.get_ephemerides(observatory)
        minutes = 0
        #count minutes where targets is at an elevation > min_elev
        for i in range(0,len(ch)):
            if ch['EL'][i] >= min_elev and ch['V'][i] <= faintest_apmag:
                minutes += 1
        #if there is an hour of observing time for this target, mark it!
        if minutes >= min_minutes:
            #start_utc = pytz.utc.localize(dt[0])
            #start_hst = start_utc.astimezone(pytz.timezone("US/Hawaii"))
            #end_utc = pytz.utc.localize(dt[1])
            #end_hst = end_utc.astimezone(pytz.timezone("US/Hawaii"))
            #print '%s to %s %s %s'%(start_hst.strftime("%Y/%m/%d %H:%M:%S"), end_hst.strftime("%Y/%m/%d %H:%M:%S"), ch['targetname'][0], minutes)
            days_available += 1
            total_minutes += minutes
        ##give the server a little rest
        #time.sleep(1)
    #is it available for each of the observation windows? record it!
    print '%s is observable for %d of the %d available sessions.'%(ch['targetname'][0], days_available, len(dts))
    if days_available == len(dts):
        print '\tspkid=%s name=%s avemin=%f'%(target.strip(), ch['targetname'][0], total_minutes/len(dts))
        f = open(fname_observables, "a")
        f.write('"%s","%s",%f\n'%(target.strip(), ch['targetname'][0], total_minutes/len(dts)))
        f.close()

Processing 2009633...
9633 Cotur (1993 UP8) is observable for 0 of the 5 available sessions.
Processing 2010648...
10648 Plancius (4089 P-L) is observable for 0 of the 5 available sessions.
Processing 2003623...
3623 Chaplin (1981 TG2) is observable for 0 of the 5 available sessions.
Processing 2006992...
6992 Minano-machi (1995 BT1) is observable for 0 of the 5 available sessions.
Processing 2053243...
53243 (1999 CW140) is observable for 0 of the 5 available sessions.
Processing 2000533...
533 Sara (1904 NZ) is observable for 0 of the 5 available sessions.
Processing 2016237...
16237 (2000 GX76) is observable for 1 of the 5 available sessions.
Processing 2006839...
6839 Ozenuma (1995 WB2) is observable for 0 of the 5 available sessions.
Processing 2004258...
4258 Ryazanov (1987 RZ2) is observable for 0 of the 5 available sessions.
Processing 2002416...
2416 Sharonov (1979 OF13) is observable for 5 of the 5 available sessions.
	spkid=2002416 name=2416 Sharonov (1979 OF13) avemin=239.0